In [1]:
import numpy as np
import pandas as pd

movie_user = pd.read_csv('link_ratings.csv')

movie_user.head()


,Unnamed: 0,movieId,imdbId,tmdbId,userId,rating,timestamp
0,0,1,114709,862.0,8,4,1013443596
1,1,1,114709,862.0,9,4,1073837180
2,2,1,114709,862.0,12,4,943912205
3,3,1,114709,862.0,20,4,1368361348
4,4,1,114709,862.0,24,4,979869938


In [2]:
movie_user = movie_user[movie_user['rating']>0]
movie_user['rating'].value_counts()

rating
4    9164499
3    8370357
5    3808511
2    3016783
1    1246154
Name: count, dtype: int64

In [3]:
import collections
movie_dict = collections.defaultdict(list)
users = movie_user['userId'].unique()
users

array([     8,      9,     12, ..., 268277,  49260, 153214], dtype=int64)

In [4]:
def partition_interval(start, end, proportions, rating_count):
    total_length = end - start
    segment_lengths = np.array([max(1, int(p * total_length)) for p in proportions])
    for i in range(len(segment_lengths)):
        if segment_lengths[i]>rating_count[i]:
            segment_lengths[i] = rating_count[i]

    if sum(segment_lengths) > total_length:
        a = segment_lengths.argmax()
        segment_lengths[a] -= (sum(segment_lengths)-total_length)
    elif sum(segment_lengths) < total_length:
        left = total_length-sum(segment_lengths)
        for i in range(len(segment_lengths)):
            x = min(left, rating_count[i]-segment_lengths[i])
            left -= x
            segment_lengths[i] += x
            if left == 0:
                break


    partition = []
    current_position = start

    for length in segment_lengths:
        partition.append((current_position, current_position + length))
        current_position += length
    result = []
    for a in partition:
        result.append(a[1]-a[0])
    return np.array(result).astype(int)



In [5]:
import csv
import jsonlines
grouped = movie_user.groupby(['userId'])
fields = ['userId', 'encode_id', 'rate_id']
for user in users:
    full_group = grouped.get_group(user)
    rating_id = np.sort(full_group['timestamp'])

    list_save = rating_id[-10:]
    group = full_group.loc[~full_group['timestamp'].isin(list_save)]
    ratings = group['rating'].unique()
    ratings = np.sort(ratings)
    rating_groups = group.groupby(['rating'])
    rating_count = rating_groups.size().to_numpy()

    num_sample_encode = partition_interval(0, 20, rating_count/rating_count.sum(), rating_count)
    num_groups_for_user = int(len(group)/20)
    for i in range(num_groups_for_user):

        list_id_encode = []
        list_id_rate = []

        max_timestamp = 0
        for k in range(len(num_sample_encode)):
            rating = ratings[k]
            rating_group = rating_groups.get_group(rating)
            samples = rating_group.sample(n=num_sample_encode[k], replace=False)
            max_timestamp = max(max_timestamp, max(samples['timestamp'].to_numpy()))
            list_id_encode += samples['tmdbId'].to_numpy().tolist()

        rate_groups = full_group.loc[full_group['timestamp']>max_timestamp]
        samples = rate_groups.sample(n=10, replace=False)
        list_id_rate += samples['tmdbId'].to_numpy().tolist()
        row_dict = {
            'userId': int(user),
            'encode_id': list_id_encode,
            'rate_id': list_id_rate
        }
        with jsonlines.open('samples.jsonl', 'a') as writer:
            writer.write(row_dict)
    

In [6]:
cnt = 0
with jsonlines.open('samples.jsonl') as reader:
    for obj in reader:
        with open('samples.csv', 'a', newline='') as csv_file:
            c = csv.DictWriter(csv_file, fieldnames=fields)
            if cnt == 0:
                c.writeheader()
            c.writerow(obj)
        cnt += 1

In [7]:
a = pd.read_csv('samples.csv')
print(len(a['userId'].unique()))

134175
